## تعبیه‌ها

در مثال قبلی، ما روی بردارهای کیسه کلمات با ابعاد بالا و طول `vocab_size` کار کردیم و به‌طور صریح بردارهای نمایشی موقعیتی با ابعاد پایین را به نمایش پراکنده یک‌داده‌ای تبدیل کردیم. این نمایش یک‌داده‌ای از نظر حافظه کارآمد نیست. علاوه بر این، هر کلمه به‌طور مستقل از کلمات دیگر در نظر گرفته می‌شود، بنابراین بردارهای کدگذاری‌شده یک‌داده‌ای شباهت‌های معنایی بین کلمات را بیان نمی‌کنند.

در این بخش، ما به بررسی مجموعه داده **News AG** ادامه خواهیم داد. برای شروع، بیایید داده‌ها را بارگذاری کنیم و برخی از تعاریف واحد قبلی را مرور کنیم.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### مفهوم تعبیه چیست؟

ایده‌ی **تعبیه** این است که کلمات را با استفاده از بردارهای متراکم با ابعاد پایین‌تر نمایش دهیم که معنای مفهومی کلمه را منعکس می‌کنند. بعداً درباره‌ی نحوه‌ی ساخت تعبیه‌های معنایی بحث خواهیم کرد، اما فعلاً می‌توانیم به تعبیه‌ها به عنوان روشی برای کاهش ابعاد بردار کلمه فکر کنیم.

بنابراین، یک لایه‌ی تعبیه یک کلمه را به عنوان ورودی دریافت می‌کند و یک بردار خروجی با اندازه‌ی مشخص `embedding_size` تولید می‌کند. به نوعی، این لایه بسیار شبیه به لایه‌ی `Dense` است، اما به جای دریافت بردار یک‌داغ (one-hot encoded) به عنوان ورودی، می‌تواند شماره‌ی کلمه را دریافت کند.

با استفاده از یک لایه‌ی تعبیه به عنوان اولین لایه در شبکه‌ی خود، می‌توانیم از مدل کیسه‌ی کلمات (bag-of-words) به مدل **کیسه‌ی تعبیه‌ها** (embedding bag) تغییر دهیم، جایی که ابتدا هر کلمه در متن خود را به تعبیه‌ی مربوطه تبدیل می‌کنیم و سپس یک تابع تجمعی مانند `sum`، `average` یا `max` را روی تمام این تعبیه‌ها محاسبه می‌کنیم.

![تصویری که یک طبقه‌بند تعبیه برای پنج کلمه‌ی دنباله را نشان می‌دهد.](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

شبکه‌ی عصبی طبقه‌بند ما شامل لایه‌های زیر است:

* لایه‌ی `TextVectorization` که یک رشته را به عنوان ورودی دریافت می‌کند و یک تنسور از شماره‌های توکن تولید می‌کند. ما یک اندازه‌ی واژگان منطقی `vocab_size` مشخص می‌کنیم و کلمات کمتر استفاده‌شده را نادیده می‌گیریم. شکل ورودی ۱ خواهد بود و شکل خروجی $n$، زیرا $n$ توکن به دست خواهیم آورد که هر کدام شامل شماره‌هایی از ۰ تا `vocab_size` هستند.
* لایه‌ی `Embedding` که $n$ شماره را دریافت می‌کند و هر شماره را به یک بردار متراکم با طول مشخص (در مثال ما ۱۰۰) کاهش می‌دهد. بنابراین، تنسور ورودی با شکل $n$ به یک تنسور $n\times 100$ تبدیل خواهد شد.
* لایه‌ی تجمع که میانگین این تنسور را در امتداد محور اول محاسبه می‌کند، یعنی میانگین تمام تنسورهای ورودی $n$ مربوط به کلمات مختلف را محاسبه می‌کند. برای پیاده‌سازی این لایه، از یک لایه‌ی `Lambda` استفاده می‌کنیم و تابع محاسبه‌ی میانگین را به آن منتقل می‌کنیم. خروجی شکل ۱۰۰ خواهد داشت و نمایش عددی کل دنباله‌ی ورودی خواهد بود.
* طبقه‌بند خطی نهایی `Dense`.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


در بخش `خلاصه`، در ستون **شکل خروجی**، بعد اول تنسور `None` نشان‌دهنده اندازه مینی‌بچ است و بعد دوم نشان‌دهنده طول دنباله توکن‌ها است. تمام دنباله‌های توکن در مینی‌بچ طول‌های متفاوتی دارند. در بخش بعدی درباره نحوه مدیریت این موضوع صحبت خواهیم کرد.

حالا بیایید شبکه را آموزش دهیم:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **توجه** داشته باشید که ما در حال ساخت بردارساز بر اساس یک زیرمجموعه از داده‌ها هستیم. این کار برای افزایش سرعت انجام می‌شود و ممکن است منجر به وضعیتی شود که همه توکن‌های متن ما در واژگان موجود نباشند. در این صورت، آن توکن‌ها نادیده گرفته می‌شوند که ممکن است باعث کاهش جزئی دقت شود. با این حال، در زندگی واقعی، یک زیرمجموعه از متن اغلب تخمین خوبی از واژگان ارائه می‌دهد.


### مدیریت اندازه‌های متغیر دنباله‌ها

بیایید بفهمیم که آموزش در مینی‌بچ‌ها چگونه انجام می‌شود. در مثال بالا، تنسور ورودی دارای بُعد ۱ است و ما از مینی‌بچ‌های ۱۲۸ تایی استفاده می‌کنیم، بنابراین اندازه واقعی تنسور برابر با $128 \times 1$ است. با این حال، تعداد توکن‌ها در هر جمله متفاوت است. اگر لایه `TextVectorization` را روی یک ورودی واحد اعمال کنیم، تعداد توکن‌های بازگشتی بسته به نحوه توکنایز شدن متن متفاوت خواهد بود:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


با این حال، زمانی که وکتورایزر را به چندین دنباله اعمال می‌کنیم، باید یک تنسور با شکل مستطیلی تولید کند، بنابراین عناصر استفاده‌نشده را با توکن PAD (که در مورد ما صفر است) پر می‌کند:


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

اینجا می‌توانیم جاسازی‌ها را ببینیم:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **توجه**: برای کاهش مقدار پر کردن، در برخی موارد منطقی است که تمام دنباله‌ها در مجموعه داده بر اساس افزایش طول (یا به طور دقیق‌تر، تعداد توکن‌ها) مرتب شوند. این کار تضمین می‌کند که هر مینی‌بچ شامل دنباله‌هایی با طول مشابه باشد.


## تعبیه‌های معنایی: Word2Vec

در مثال قبلی، لایه تعبیه یاد گرفت که کلمات را به نمایش‌های برداری تبدیل کند، اما این نمایش‌ها معنای معنایی نداشتند. خوب است که یک نمایش برداری یاد بگیریم به‌طوری‌که کلمات مشابه یا مترادف‌ها به بردارهایی تبدیل شوند که از نظر فاصله برداری (مثلاً فاصله اقلیدسی) به یکدیگر نزدیک باشند.

برای انجام این کار، باید مدل تعبیه خود را با استفاده از تکنیکی مانند [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) بر روی یک مجموعه بزرگ از متون پیش‌آموزش دهیم. این تکنیک بر اساس دو معماری اصلی است که برای تولید نمایش توزیع‌شده کلمات استفاده می‌شوند:

- **کیسه کلمات پیوسته** (CBoW)، که در آن مدل را آموزش می‌دهیم تا یک کلمه را از متن اطرافش پیش‌بینی کند. با داشتن ngram به شکل $(W_{-2},W_{-1},W_0,W_1,W_2)$، هدف مدل این است که $W_0$ را از $(W_{-2},W_{-1},W_1,W_2)$ پیش‌بینی کند.
- **اسکیپ-گرام پیوسته** برعکس CBoW است. مدل از پنجره کلمات متنی اطراف استفاده می‌کند تا کلمه فعلی را پیش‌بینی کند.

CBoW سریع‌تر است، و در حالی که اسکیپ-گرام کندتر است، در نمایش کلمات نادر بهتر عمل می‌کند.

![تصویری که هر دو الگوریتم CBoW و اسکیپ-گرام را برای تبدیل کلمات به بردارها نشان می‌دهد.](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

برای آزمایش تعبیه Word2Vec که بر روی مجموعه داده Google News پیش‌آموزش داده شده است، می‌توانیم از کتابخانه **gensim** استفاده کنیم. در زیر کلماتی را که بیشترین شباهت را به 'neural' دارند پیدا می‌کنیم.

> **توجه:** وقتی برای اولین بار بردارهای کلمات را ایجاد می‌کنید، دانلود آن‌ها ممکن است کمی زمان‌بر باشد!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


ما همچنین می‌توانیم بردار جاسازی را از کلمه استخراج کنیم تا در آموزش مدل طبقه‌بندی استفاده شود. این جاسازی دارای ۳۰۰ مؤلفه است، اما در اینجا فقط ۲۰ مؤلفه اول بردار را برای وضوح نشان می‌دهیم:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

نکته عالی درباره تعبیه‌های معنایی این است که می‌توانید رمزگذاری برداری را بر اساس معنا دستکاری کنید. برای مثال، می‌توانیم بخواهیم کلمه‌ای پیدا کنیم که نمایش برداری آن تا حد ممکن به کلمات *پادشاه* و *زن* نزدیک باشد و تا حد ممکن از کلمه *مرد* دور باشد:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

یک مثال بالا از جادوی داخلی GenSym استفاده می‌کند، اما منطق زیرین در واقع بسیار ساده است. نکته جالب درباره تعبیه‌ها این است که می‌توانید عملیات برداری معمولی را روی بردارهای تعبیه انجام دهید و این عملیات‌ها بازتابی از عملیات روی **معانی** کلمات خواهند بود. مثال بالا را می‌توان به صورت عملیات برداری بیان کرد: ما برداری را محاسبه می‌کنیم که متناظر با **پادشاه-مرد+زن** باشد (عملیات `+` و `-` روی نمایش‌های برداری کلمات متناظر انجام می‌شود) و سپس نزدیک‌ترین کلمه در فرهنگ لغت به آن بردار را پیدا می‌کنیم:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **توجه**: ما مجبور شدیم ضرایب کوچکی به بردارهای *مرد* و *زن* اضافه کنیم - سعی کنید آنها را حذف کنید تا ببینید چه اتفاقی می‌افتد.

برای یافتن نزدیک‌ترین بردار، از ابزارهای TensorFlow استفاده می‌کنیم تا یک بردار فاصله بین بردار ما و تمام بردارهای موجود در واژگان محاسبه کنیم، و سپس با استفاده از `argmin` شاخص کلمه حداقلی را پیدا کنیم.


در حالی که Word2Vec به نظر می‌رسد راهی عالی برای بیان معنای کلمات باشد، معایب زیادی دارد، از جمله موارد زیر:

* هم مدل‌های CBoW و هم skip-gram **تعبیه‌های پیش‌بینی‌کننده** هستند و فقط زمینه محلی را در نظر می‌گیرند. Word2Vec از زمینه جهانی بهره نمی‌برد.
* Word2Vec به **ریشه‌شناسی کلمات** توجه نمی‌کند، یعنی این واقعیت که معنای کلمه می‌تواند به بخش‌های مختلف آن، مانند ریشه، وابسته باشد.

**FastText** تلاش می‌کند محدودیت دوم را برطرف کند و بر اساس Word2Vec عمل می‌کند، با یادگیری نمایش‌های برداری برای هر کلمه و n-gram‌های کاراکتری که در هر کلمه یافت می‌شوند. مقادیر این نمایش‌ها در هر مرحله آموزش به یک بردار واحد میانگین‌گیری می‌شوند. در حالی که این کار محاسبات اضافی زیادی به پیش‌آموزش اضافه می‌کند، اما به تعبیه‌های کلمات امکان می‌دهد اطلاعات زیرکلمه‌ای را رمزگذاری کنند.

روش دیگری به نام **GloVe** از رویکرد متفاوتی برای تعبیه‌های کلمات استفاده می‌کند که بر اساس تجزیه ماتریس زمینه-کلمه است. ابتدا یک ماتریس بزرگ ایجاد می‌کند که تعداد وقوع کلمات در زمینه‌های مختلف را شمارش می‌کند، و سپس تلاش می‌کند این ماتریس را در ابعاد پایین‌تر نمایش دهد به گونه‌ای که کمترین میزان از دست دادن بازسازی را داشته باشد.

کتابخانه gensim از این تعبیه‌های کلمات پشتیبانی می‌کند و شما می‌توانید با تغییر کد بارگذاری مدل در بالا، با آنها آزمایش کنید.


## استفاده از تعبیه‌های از پیش آموزش‌دیده‌شده در Keras

می‌توانیم مثال بالا را تغییر دهیم تا ماتریس در لایه تعبیه خود را با تعبیه‌های معنایی، مانند Word2Vec، از پیش پر کنیم. احتمالاً واژگان تعبیه از پیش آموزش‌دیده‌شده و متن موجود در مجموعه داده با یکدیگر مطابقت نخواهند داشت، بنابراین باید یکی را انتخاب کنیم. در اینجا دو گزینه ممکن را بررسی می‌کنیم: استفاده از واژگان tokenizer و استفاده از واژگان تعبیه‌های Word2Vec.

### استفاده از واژگان tokenizer

هنگام استفاده از واژگان tokenizer، برخی از کلمات موجود در واژگان دارای تعبیه‌های Word2Vec متناظر خواهند بود و برخی دیگر وجود نخواهند داشت. با توجه به اینکه اندازه واژگان ما `vocab_size` است و طول بردار تعبیه Word2Vec برابر با `embed_size` است، لایه تعبیه با یک ماتریس وزن به شکل `vocab_size`$\times$`embed_size` نمایش داده می‌شود. این ماتریس را با مرور واژگان پر خواهیم کرد:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


برای کلماتی که در واژگان Word2Vec وجود ندارند، می‌توانیم آن‌ها را به صورت صفر باقی بگذاریم یا یک بردار تصادفی تولید کنیم.

اکنون می‌توانیم یک لایه تعبیه با وزن‌های از پیش آموزش‌دیده تعریف کنیم:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **توجه**: دقت کنید که هنگام ایجاد `Embedding` مقدار `trainable=False` تنظیم شده است، به این معنی که لایه Embedding دوباره آموزش داده نمی‌شود. این ممکن است باعث کاهش جزئی دقت شود، اما سرعت آموزش را افزایش می‌دهد.

### استفاده از واژگان embedding

یکی از مشکلات روش قبلی این است که واژگان استفاده‌شده در TextVectorization و Embedding متفاوت هستند. برای حل این مشکل، می‌توانیم یکی از راه‌حل‌های زیر را استفاده کنیم:
* مدل Word2Vec را با واژگان خودمان دوباره آموزش دهیم.
* مجموعه داده خود را با واژگان مدل Word2Vec از پیش آموزش‌داده‌شده بارگذاری کنیم. واژگان مورد استفاده برای بارگذاری مجموعه داده را می‌توان هنگام بارگذاری مشخص کرد.

روش دوم به نظر ساده‌تر می‌آید، پس بیایید آن را پیاده‌سازی کنیم. ابتدا یک لایه `TextVectorization` با واژگان مشخص‌شده که از embedding‌های Word2Vec گرفته شده است، ایجاد می‌کنیم:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

کتابخانه تعبیه کلمات gensim شامل یک تابع کاربردی به نام `get_keras_embeddings` است که به طور خودکار لایه تعبیه‌های Keras مربوطه را برای شما ایجاد می‌کند.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


یکی از دلایلی که دقت بالاتری مشاهده نمی‌کنیم این است که برخی از کلمات موجود در مجموعه داده ما در واژگان پیش‌آموزش GloVe وجود ندارند و بنابراین عملاً نادیده گرفته می‌شوند. برای غلبه بر این مشکل، می‌توانیم تعبیه‌های خود را بر اساس مجموعه داده خود آموزش دهیم.


## تعبیه‌های متنی

یکی از محدودیت‌های اصلی تعبیه‌های از پیش آموزش‌دیده شده سنتی مانند Word2Vec این است که، حتی اگر بتوانند بخشی از معنای یک کلمه را درک کنند، نمی‌توانند بین معانی مختلف آن تمایز قائل شوند. این موضوع می‌تواند در مدل‌های پایین‌دستی مشکلاتی ایجاد کند.

برای مثال، کلمه «play» در این دو جمله معانی متفاوتی دارد:
- من به یک **نمایش** در تئاتر رفتم.
- جان می‌خواهد با دوستانش **بازی** کند.

تعبیه‌های از پیش آموزش‌دیده شده‌ای که درباره‌شان صحبت کردیم، هر دو معنای کلمه «play» را در یک تعبیه مشابه نمایش می‌دهند. برای غلبه بر این محدودیت، نیاز داریم تعبیه‌هایی بر اساس **مدل زبانی** بسازیم، مدلی که بر روی یک مجموعه داده بزرگ از متن آموزش دیده و *می‌داند* که چگونه کلمات می‌توانند در زمینه‌های مختلف کنار هم قرار بگیرند. بحث درباره تعبیه‌های متنی فراتر از محدوده این آموزش است، اما در واحد بعدی، هنگام صحبت درباره مدل‌های زبانی، به آن‌ها بازخواهیم گشت.



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
